In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
class PUpsamplend(nn.Module):
    def __init__(self, scale_factor, n):
        super(PUpsamplend, self).__init__()
        self.scale_factor = scale_factor
        self.n = n
        self.conv_transposes = {1: F.conv_transpose1d, 2: F.conv_transpose2d, 3: F.conv_transpose3d}

    def forward(self, x, shifts):
        w = x.new_zeros(*((self.scale_factor,)*self.n))
        w.view(-1)[0] = 1
        output = self.conv_transposes[self.n](x, w.expand(x.size(1), 1, *((self.scale_factor,)*self.n)), stride = self.scale_factor, groups = x.size(1))
        for i in range(self.n):
            indices = (torch.arange(output.shape[2+i])[None, None, :].repeat(1, 2, 1) - shifts[:, i][:, None, None]) % output.shape[2+i]
            output = torch.gather(output, 2+i, indices)
        return output